# Higher order functions
* Higher order functions are functions that operate on complex data types such as arrays and maps
* They allow you to pass functions as arguments (e.g., lambda expressions), apply transformations and return arrays or maps
* They are extremely useful for manipulating arrays without exploding them.

## Commonly used higher order array functions
* transform
* filter
* exists
* aggregate

### Syntax
`<function_name>` (`array_column`, `lambda_expression`)

`lambda_expression: element -> expression`


In [0]:
%sql
create or replace temp view order_items 
as
select *
from values
  (1, array('smartphone', 'laptop', 'monitor')),
  (2, array('headphones', 'mouse', 'smartwatch')),
  (3, array('keyboard', 'mouse'))
as orders(order_id, items);


In [0]:
%sql
select *
from order_items;

In [0]:
%sql
-- transform function to convert all array elements to uppercase
select
  order_id,
  transform(items, x -> upper(x)) as items
from order_items

In [0]:
%sql
-- filter items that contains the string smart
select
  order_id,
  filter(items, x -> x like '%smart%') as smart_items
from order_items

In [0]:
%sql
-- exists
-- function to check whether order contains any item monitor
select 
  order_id, 
  exists(items, x -> x = 'monitor') as contains_monitor
from order_items


In [0]:
%sql
create or replace temp view order_items as 
select * from values
  (1, array(
    named_struct('name', 'smartphone', 'price', 1000),
    named_struct('name', 'laptop', 'price', 2000),
    named_struct('name', 'monitor', 'price', 500)
  )),
  (2, array(
    named_struct('name', 'headphones', 'price', 100),
    named_struct('name', 'mouse', 'price', 50),
    named_struct('name', 'smartwatch', 'price', 200)
  )),
  (3, array(
    named_struct('name', 'keyboard', 'price', 100),
    named_struct('name', 'mouse', 'price', 50)
  ))
as orders(order_id, items);

select * from order_items;


In [0]:
%sql
-- aggregate 
-- convert all items to uppercase and add 10% tax
-- calculate the total order amount for each of the order
with transformed_items as
(
  select
    order_id, 
    transform(items, x -> named_struct('name', upper(x.name),
                                      'price', round(x.price * 1.1, 2))) as items
  from order_items
)
select
  order_id,
  aggregate(
    items, 
    cast(0.0 as double), 
    (acc, x) -> acc + coalesce(x.price, 0.0)
    ) as total_price
from transformed_items

## Commonly used higher order map functions
* transform_values
* transform_keys
* map_filter

### Syntax
`<function_name>` (`map_column`, `lambda_expression`)

`lambda_expression: (key, value) -> expression`

In [0]:
%sql
create or replace temp view order_item_prices as
select * from values
  (1, map('smartphone', 1000, 'laptop', 2000, 'monitor', 500)),
  (2, map('headphones', 100, 'mouse', 50, 'smartwatch', 200)),
  (3, map('keyboard', 100, 'mouse', 50))
as orders(order_id, item_prices)

In [0]:
%sql
select * from order_item_prices

In [0]:
%sql
-- transform_keys function
-- convert all the item names to uppercase 
select
  order_id,
  transform_keys(item_prices, (k, v) -> upper(k)) as items
from order_item_prices

In [0]:
%sql
-- transform_values
-- apply 10% tax on item prices
select 
  order_id,
  transform_values(item_prices, (k, v) -> round(v*1.1, 2)) as items
from order_item_prices;

In [0]:
%sql
-- map filter
-- filter items with prices above 500
select 
  order_id,
  map_filter(item_prices, (k, v) -> v > 500) as items
from order_item_prices